In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Project root
cwd = os.getcwd()
project_root = os.path.abspath("..") if os.path.basename(cwd) == "notebooks" else cwd
sys.path.append(os.path.join(project_root, "src"))

OUTPUT_DIR = os.path.join(project_root, "data/processed")

print("✅ Libraries imported successfully!")

## 1. Load dữ liệu

In [ ]:
# Load customer clusters data
df_clusters = pd.read_csv(os.path.join(OUTPUT_DIR, "customer_clusters_from_rules.csv"))

print(f"📊 Loaded {len(df_clusters):,} customers")
print(f"   Columns: {df_clusters.columns.tolist()}")
df_clusters.head()

In [ ]:
# Lấy RFM features
rfm_cols = ['Recency', 'Frequency', 'Monetary']
X_rfm = df_clusters[rfm_cols].values

# Chuẩn hóa
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_rfm)

print(f"✅ RFM features shape: {X_scaled.shape}")

## 2. Hierarchical Clustering với các linkage methods

In [ ]:
# Lấy sample để vẽ dendrogram (full data quá lớn)
# Dendrogram với 4000 khách hàng sẽ rất khó nhìn
# Ta sẽ lấy random sample hoặc vẽ truncated dendrogram

np.random.seed(42)
sample_size = min(500, len(X_scaled))  # Lấy tối đa 500 để dendrogram dễ nhìn
sample_idx = np.random.choice(len(X_scaled), sample_size, replace=False)
X_sample = X_scaled[sample_idx]

print(f"📊 Sử dụng {sample_size} khách hàng để vẽ dendrogram")

In [ ]:
# Tính linkage matrix cho các methods khác nhau
methods = ['ward', 'complete', 'average', 'single']
linkage_matrices = {}

for method in methods:
    print(f"🔄 Computing {method} linkage...", end=" ")
    linkage_matrices[method] = linkage(X_sample, method=method)
    print("Done!")

print("\n✅ All linkage matrices computed!")

## 3. Vẽ Dendrogram - So sánh các linkage methods

In [ ]:
# Vẽ 4 dendrograms để so sánh
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
axes = axes.flatten()

method_names = {
    'ward': 'Ward (minimize variance)',
    'complete': 'Complete (max distance)',
    'average': 'Average (UPGMA)',
    'single': 'Single (min distance)'
}

for idx, method in enumerate(methods):
    ax = axes[idx]
    
    # Vẽ dendrogram truncated
    dendrogram(
        linkage_matrices[method],
        ax=ax,
        truncate_mode='level',
        p=5,  # Hiển thị 5 levels cuối
        leaf_rotation=90,
        leaf_font_size=8,
        color_threshold=0.7 * max(linkage_matrices[method][:, 2]),
        above_threshold_color='gray'
    )
    
    ax.set_title(f'Dendrogram - {method_names[method]}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Samples (or cluster size)')
    ax.set_ylabel('Distance')
    ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('🌳 So Sánh Các Linkage Methods trong Hierarchical Clustering', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'dendrogram_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Saved: dendrogram_comparison.png")

## 4. Dendrogram chi tiết với Ward linkage

In [ ]:
# Vẽ dendrogram chi tiết cho Ward (phổ biến nhất)
plt.figure(figsize=(20, 10))

# Dendrogram với color threshold để highlight clusters
Z = linkage_matrices['ward']

# Tìm threshold tốt để cắt thành K clusters
# Dựa vào khoảng cách lớn nhất giữa 2 lần merge liên tiếp
distances = Z[:, 2]
diff = np.diff(distances)
elbow_idx = np.argmax(diff) + 1  # Vị trí "elbow"
threshold = (distances[elbow_idx] + distances[elbow_idx - 1]) / 2

dendrogram(
    Z,
    truncate_mode='level',
    p=8,
    leaf_rotation=90,
    leaf_font_size=9,
    color_threshold=threshold,
    show_contracted=True
)

# Vẽ đường cắt
plt.axhline(y=threshold, color='red', linestyle='--', linewidth=2, 
            label=f'Threshold = {threshold:.2f}')

plt.title('🌳 Dendrogram - Ward Linkage (Hierarchical Clustering)\n'
          'Đường đỏ: Ngưỡng cắt để xác định số clusters', 
          fontsize=14, fontweight='bold')
plt.xlabel('Samples (số trong ngoặc = số khách hàng bị gộp)', fontsize=12)
plt.ylabel('Distance (Ward variance)', fontsize=12)
plt.legend(loc='upper right', fontsize=11)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'dendrogram_ward.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Threshold tự động: {threshold:.2f}")
print("✅ Saved: dendrogram_ward.png")

## 5. Xác định số clusters từ Dendrogram

In [ ]:
# Vẽ biểu đồ khoảng cách merge để tìm "elbow"
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Merge distances
last_merges = Z[-20:, 2]  # 20 lần merge cuối
axes[0].plot(range(1, len(last_merges) + 1), last_merges[::-1], 'bo-', markersize=8)
axes[0].set_xlabel('Số clusters', fontsize=12)
axes[0].set_ylabel('Merge Distance', fontsize=12)
axes[0].set_title('📉 Merge Distance vs Số Clusters\n(Tìm điểm "elbow")', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(range(1, len(last_merges) + 1))

# Plot 2: Difference in merge distances
diff_merges = np.diff(last_merges[::-1])
axes[1].bar(range(2, len(diff_merges) + 2), diff_merges, color='steelblue', edgecolor='black')
axes[1].set_xlabel('Số clusters', fontsize=12)
axes[1].set_ylabel('Δ Distance', fontsize=12)
axes[1].set_title('📊 Độ thay đổi khoảng cách merge\n(Cột cao nhất = K tối ưu)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Highlight best K
best_k_idx = np.argmax(diff_merges)
suggested_k = best_k_idx + 2
axes[1].axvline(x=suggested_k, color='red', linestyle='--', linewidth=2, label=f'Suggested K = {suggested_k}')
axes[1].legend()

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'dendrogram_elbow.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n🏆 Số clusters gợi ý từ Dendrogram: K = {suggested_k}")
print("✅ Saved: dendrogram_elbow.png")

## 6. So sánh Dendrogram với kết quả K-Means

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score

# Lấy labels từ K-Means (đã có trong dataset)
if 'Cluster' in df_clusters.columns:
    kmeans_labels = df_clusters['Cluster'].values
    k_from_kmeans = len(np.unique(kmeans_labels))
    
    # Chạy Agglomerative với cùng số K
    agg = AgglomerativeClustering(n_clusters=k_from_kmeans, linkage='ward')
    agg_labels = agg.fit_predict(X_scaled)
    
    # So sánh
    print("="*60)
    print("SO SÁNH K-MEANS VỚI HIERARCHICAL CLUSTERING (Ward)")
    print("="*60)
    print(f"\n📊 Số clusters: K = {k_from_kmeans}")
    print(f"\n  K-Means:")
    print(f"    - Silhouette Score: {silhouette_score(X_scaled, kmeans_labels):.4f}")
    
    print(f"\n  Agglomerative (Ward):")
    print(f"    - Silhouette Score: {silhouette_score(X_scaled, agg_labels):.4f}")
    
    # Adjusted Rand Index (đo độ tương đồng giữa 2 cách phân cụm)
    ari = adjusted_rand_score(kmeans_labels, agg_labels)
    print(f"\n  Adjusted Rand Index (độ tương đồng): {ari:.4f}")
    if ari > 0.8:
        print("    → Hai cách phân cụm RẤT GIỐNG nhau!")
    elif ari > 0.5:
        print("    → Hai cách phân cụm TƯƠNG ĐỐI GIỐNG nhau")
    else:
        print("    → Hai cách phân cụm KHÁC nhau đáng kể")
else:
    print("⚠️ Không tìm thấy cột 'Cluster' trong dataset")

## 7. Tổng kết

In [ ]:
print("="*60)
print("🌳 TỔNG KẾT DENDROGRAM ANALYSIS")
print("="*60)

print("""
📊 DENDROGRAM LÀ GÌ?
   - Biểu đồ cây thể hiện quá trình gộp clusters trong Hierarchical Clustering
   - Trục Y: Khoảng cách/độ khác biệt giữa các clusters
   - Cắt ngang cây tại một mức → Xác định số clusters

📋 CÁC LINKAGE METHODS:
   1. Ward: Minimize tổng variance trong cluster (phổ biến nhất)
   2. Complete: Dựa trên khoảng cách MAX giữa các điểm
   3. Average: Dựa trên khoảng cách TRUNG BÌNH
   4. Single: Dựa trên khoảng cách MIN (hay tạo "chaining effect")

🎯 ƯU ĐIỂM CỦA DENDROGRAM:
   ✅ Không cần chọn K trước
   ✅ Cho thấy cấu trúc phân cấp của dữ liệu
   ✅ Có thể cắt ở nhiều mức để thử các số K khác nhau

⚠️ NHƯỢC ĐIỂM:
   ❌ Chậm với dữ liệu lớn (O(n³))
   ❌ Không thể thay đổi sau khi merge
   ❌ Với dữ liệu >1000 points, dendrogram khó nhìn

✅ FILES ĐÃ LƯU:
   - dendrogram_comparison.png (so sánh 4 methods)
   - dendrogram_ward.png (chi tiết Ward)
   - dendrogram_elbow.png (tìm K tối ưu)
""")

print("="*60)